# Boosting<br><sub>Author: Greg Holste<br></sub>

## Introduction

Boosting is an **ensemble** technique for classification or regression; ensemble methods in general work by fitting many models to the data and somehow aggregating their predictions. The broad idea behind boosting is to sequentially train many *weak learners* (models barely better than random guessing), adaptively giving more weight to misclassified examples at each iteration. This way, the boosted model learns to "pay more attention" to the hardest-to-learn training observations. The learners in a boosted model are typically classification/regression trees, but in principle boosting can be applied to any model.

## AdaBoost.M1

This is the first boosting algorithm for binary classification. Let our training set consist of $n$ pairs $\{({\bf x_i}, y_i)\}_{i=1}^n$, where each ${\bf x_i}$ is a vector of (continuous or discrete) features and each $y_i \in \{-1, 1\}$ is a binary label. For convenience, $I(\cdot) = \begin{cases} 1, \cdot \text{ is true}\\ 0, \cdot \text{ is false}\end{cases}$ is the *indicator function*. The algorithm is as follows.

1. Initialize $w_i \gets \frac{1}{n}$ for $i = 1, \dots, n$
2. For $m = 1, \dots, M$ do
    1. Fit classifier $G_m({\textbf x})$ to data with weights $w_i, i = 1, \dots, n$
    2. Find weighted error $err_m = \frac{\sum_{i=1}^n w_i I(y_i \neq G_m({\bf x}))}{\sum_{i=1}^n w_i}$
    3. Find $\alpha_m = log\left(\frac{1 - err_m}{err_m}\right)$
    4. Update weights via $w_i \gets w_i \cdot e^{\alpha_m I(y_i \neq G_m({\bf x})}$
3. Output boosted classifier $G({\bf x}) = sign\{\sum_{m=1}^M \alpha_m G_m({\bf x})\}$

To start, we give equal weight to each training observation and train a classifier. We then find a weighted measure of training error for the classifier $G_m({\bf x})$ (at first, $err_m$ is simply the misclassification rate). In step 2D, we now update update the weights for *only the misclassified observations*; that is, **each misclassified observation has its weight scaled by $e^{\alpha_m}$, thus increasing its influence on the next classifier**. After repeating this process $M$ times, we finally output a weighted sum of our $M$ weak learners, where the weights $\alpha_m, m = 1, \dots, M$ correspond to the predictive ability of each learner.

Suppose we want to apply AdaBoost with classification trees. Then two basic hyperparameters of interest are $M$, the number of learners in our ensemble, and $J_m, m=1,\dots,M$, the "depth" of each tree. Interestingly, often shallower trees see better overall performance; using $J_m=1$ for all $m=1,\dots,M$ -- creating an ensemble of "stumps" (trees with a single decision node) -- is often a good starting point. That being said, $M$ and $J_m$ should be tuned like any other hyperparameter via *cross-validation* or a related method. As a final note, AdaBoost can be expanded to accomodate multiple output classes and even probabilities of class membership. 

## Gradient Boosted Trees

Gradient boosting provides a powerful, generalized approach to ensembling learners, most often decision trees. First observe that we can formally represent a decision tree as $T({\bf x}) = \sum_{j=1}^J \gamma_j I({\bf x} \in R_j)$, where $\gamma_j$ is the constant predicted value associated with terminal node region $R_j$.

1. Initialize $f_0({\bf x}) = \arg\!\min_{\gamma} \sum_{i=1}^n L(y_i, \gamma)$
2. For $m = 1, \dots, M$ do
    1. Find $r_{im} = -\left[\frac{\partial L(y_i, f_m({\bf x_i}))}{\partial f_m({\bf x_i})}\right]$ for all $i = 1, \dots, n$
    2. Fit regression tree to negative gradients $r_{im}$
    3. Update $f_m({\bf x}) \gets f_{m-1}({\bf x}) + \lambda \sum_{j=1} \gamma_{jm} I({\bf x} \in R_{jm})$
3. Output boosted model $\hat{f}({\bf x}) = f_M({\bf x})$

While may seem heavy on notation, the steps are actually quite simple. In step 1, we initalize our model with the best constant value (that which minimizes our loss). In step 2A, we find the negative gradient of our loss wrt each prediction. Observe that **when we use mean squared-error loss $L = -\frac{1}{2}(y_i - f({\bf x_i}))^2$, each $r_{im}$ is simply the *residual* $y_i - f({\bf x_i})$**; for this reason, these $r_{im}, i = 1, \dots, n$ are aptly called "pseudoresiduals." We then fit a regression tree using these psuedoresiduals as labels (instead of our true training labels); in principle, we can fit any model to our negative gradients -- not just a decision tree. We then add a shrunken version of this tree (shrunk by learning rate $\lambda$) to $f_0({\bf x})$ and repeat this process $M$ times.

Suppose we want to use this algorithm for a classification in which our output has $C$ unordered levels. Then we could use the multinomial deviance loss $L = -\sum_{c \in C} I(y_i = c)\ log(p_c({\bf x}))$, where $p_c({\bf x}) = \frac{e^{f_c({\bf x})}}{\sum_{l \in C} e^{f_l({\bf x})}}$ is the *softmax* function which maps inputs to probabilities (that sum to $1$). We would then repeat steps 2A-2C $C$ times -- where our target is an indicator for each level of output $y$ -- and proceed as normal, noting that the negative gradients in Step 2A are given by $r_{imc} = I(y_i = c) - p_c({\bf x_i})$ for $i = 1, \dots, n$ and $c \in C$.

Now we have an additional hyperparameter $\lambda$ to consider; there is an inherent trade-off between the number of learners $M$ and the learning rate (or "step size") $\lambda$, and these should be tuned accordingly. As a final note, observe that gradient boosted regression trees (GBRT) is more general than AdaBoost because it can do regression *or* classification and can accomodate any loss function. Furthermore, when using GBRT to perform binary classification with an exponential loss function, this algorithm reduces to AdaBoost.M1.

## AdaBoost: Toy Example

Let's consider a simple application of these boosting methods to a toy binary classification problem. We will show that boosting outperforms a single tree and that our implementation of AdaBoost (in `boosting.py`) matches that of sk-learn. Here is our data...

In [1]:
from boosting import *
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, load_boston
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
y[y == 0] = -1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

df = pd.DataFrame(np.concatenate([X, y[:, np.newaxis]], axis=-1))
df.columns = ['$X_1$', '$X_2$', '$X_3$', '$X_4$', '$Y$']
df

,$X_1$,$X_2$,$X_3$,$X_4$,$Y$
0,-1.668532,-1.299013,0.274647,-0.603620,-1.0
1,-2.972883,-1.088783,0.708860,0.422819,-1.0
2,-0.596141,-1.370070,-3.116857,0.644452,-1.0
3,-1.068947,-1.175057,-1.913743,0.663562,-1.0
4,-1.305269,-0.965926,-0.154072,1.193612,-1.0
...,...,...,...,...,...
995,-0.383660,0.952012,-1.738332,0.707135,1.0
996,-0.120513,1.172387,0.030386,0.765002,1.0
997,0.917112,1.105966,0.867665,-2.256250,1.0
998,0.100277,1.458758,-0.443603,-0.670023,1.0


Let us first look at results when fitting a single classification tree to the training data with minimum terminal node size $2$.

In [2]:
DT = DecisionTreeClassifier(min_samples_leaf=2).fit(X_train, y_train)
print(f"{round(DT.score(X_train, y_train) * 100, 3)}% training accuracy")
print(f"{round(DT.score(X_test, y_test) * 100, 3)}% test accuracy")

98.133% training accuracy
93.2% test accuracy


Now we'll use AdaBoost with $M=500$ and $J_m=1$ for all $m=1,\dots,m$ (comparing our custom implementation with sklearn's). 

In [3]:
AB = AdaBoostClassifier(n_estimators=500, algorithm='SAMME').fit(X_train, y_train)

AB2 = AdaBoost(X_train, y_train, M=500, J=1)
AB2.fit()

print(f"{round(AB.score(X_train, y_train) * 100, 3)}% training accuracy with sk-learn AdaBoost")
print(f"{round(acc(y_train, AB2.predict(X_train)) * 100, 3)}% training accuracy with custom AdaBoost")
print(f"{round(AB.score(X_test, y_test) * 100, 3)}% test accuracy with sk-learn AdaBoost")
print(f"{round(acc(y_test, AB2.predict(X_test)) * 100, 3)}% test accuracy with custom AdaBoost")

98.133% training accuracy with sk-learn AdaBoost
98.133% training accuracy with custom AdaBoost
95.6% test accuracy with sk-learn AdaBoost
95.6% test accuracy with custom AdaBoost


## GBRT: Toy Example

Now let's consider a quick example applying gradient boosted regression trees (GBRT) to the Boston house pricing dataset (from the UCI Machine Learning Repository).

In [4]:
boston = load_boston()
X = boston.data
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

df = pd.DataFrame(np.concatenate([X, y[:, np.newaxis]], axis=-1))
df.columns = ["$X_{" + str(i) + "}$" for i in range(1, X.shape[1] + 1)] + ["$Y$"]
df

,$X_{1}$,$X_{2}$,$X_{3}$,$X_{4}$,$X_{5}$,$X_{6}$,$X_{7}$,$X_{8}$,$X_{9}$,$X_{10}$,$X_{11}$,$X_{12}$,$X_{13}$,$Y$
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


First let's apply a single decision tree with mean squared-error (MSE) loss and minimum node size $1$.

In [5]:
DT = DecisionTreeRegressor(min_samples_leaf=1, random_state=0).fit(X_train, y_train)
print(f"R^2 = {round(DT.score(X_train, y_train), 3)} on training set")
print(f"R^2 = {round(DT.score(X_test, y_test), 3)} on test set")

R^2 = 1.0 on training set
R^2 = 0.613 on test set


Now let us use GBRT with $M=1000$, $J_m=2, i=1,\dots,M$, and $\lambda = 0.1$, again comparing our implementation with that of sklearn. (Note: While there is no randomness in the GBRT algorithm as presented here, there could be slight differences in performance due to lack of precision (when summing the many small gradients).)

In [7]:
GB = GradientBoostingRegressor(n_estimators=1000, max_depth=2, learning_rate=0.1, criterion='mse',
                               init=DecisionTreeRegressor(max_depth=1).fit(X_train, y_train))
GB = GB.fit(X_train, y_train)

GB2 = GradientBoostRegressor(X_train, y_train, M=1000, J=2, lam=0.1)
GB2.fit()

print(f"R^2 = {round(GB.score(X_train, y_train), 3)} on training set")
print(f"R^2 = {round(R2(y_train, GB2.predict(X_train)), 3)} on training set")
print(f"R^2 = {round(GB.score(X_test, y_test), 3)} on test set")
print(f"R^2 = {round(R2(y_test, GB2.predict(X_test)), 3)} on test set")

R^2 = 0.998 on training set
R^2 = 0.998 on training set
R^2 = 0.693 on test set
R^2 = 0.693 on test set
